# Generate Dataset Contains Space, English, Greek, Symbols

## Combination of Greek Words

In [49]:
import pandas as pd
import csv
greek_corpus = pd.read_csv('Greek_corpus.txt', sep="\t", header=None, quoting=csv.QUOTE_NONE)

In [50]:
df_dict = pd.read_csv('greek_eng_dict.txt', sep="\t", header=None, quoting=csv.QUOTE_NONE)
dictionary = pd.read_csv('greek_eng_dict.txt', sep="\t", header=None, quoting=csv.QUOTE_NONE)
df_dict = df_dict.iloc[110:]
df_dict = df_dict.reset_index(drop=True)

In [51]:
greek_corpus

,0
0,και
1,να
2,το
3,του
4,την
...,...
143955,ΚΑΚΟΗΘΟΥΣ
143956,ΗΛΙΑΧΤΙΔΑ
143957,ΠΡΕΣΒΕΥΕΙΣ
143958,ΕΦΑΜΙΛΛΗ


In [52]:
df_dict

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


## Data Cleansing for Greek Dataset

In [53]:
dict_list = list(dictionary[0])

remove_list = []

for i, txt in enumerate(greek_corpus[0]):
    try:
        for j in txt:
            if j in dict_list:
                pass
            else:
                remove_list.append(i)
                continue
    except:
        remove_list.append(i)

In [54]:
greek_corpus_avalible = greek_corpus.drop(index=remove_list)
greek_corpus_avalible = greek_corpus_avalible.reset_index(drop=True)
greek_corpus_avalible

,0
0,και
1,να
2,το
3,του
4,την
...,...
71977,ΔΠΘ
71978,ΠΣΔ
71979,ΛΔ
71980,ΣΔ


## Data Cleansing for English Dataset

In [55]:
eng_corpus = pd.read_csv('eng.wordlist', sep="\t", header=None, quoting=csv.QUOTE_NONE)

In [56]:
eng_corpus

,0
0,the
1,of
2,and
3,to
4,a
...,...
439145,Cycling:
439146,variation).
439147,people.'
439148,Minhtu


In [57]:
dict_list = list(dictionary[0])

remove_list = []

for i, txt in enumerate(eng_corpus[0]):
    try:
        for j in txt:
            if j in dict_list:
                pass
            else:
                remove_list.append(i)
                continue
    except:
        remove_list.append(i)

In [58]:
eng_corpus_avalible = eng_corpus.drop(index=remove_list)
eng_corpus_avalible = eng_corpus_avalible.reset_index(drop=True)
eng_corpus_avalible

,0
0,the
1,of
2,and
3,to
4,a
...,...
399651,Cycling:
399652,variation).
399653,people.'
399654,Minhtu


In [59]:
with open('English_corpus.txt', 'w') as z:

    for i in eng_corpus_avalible[0]:
        z.write(i)
        z.write('\n')

## Generate Combined Text

In [60]:
import random

n_sample = 100

for i in range(n_sample):

    # Whether the text is combined by english and greek

    greek_eng = random.randint(0,1)

    if greek_eng:

        # Set english first or greek first

        eng_first = random.randint(0,1)

        if eng_first:

            # Pick Words in Corpus

            f = random.randint(0,len(eng_corpus_avalible)-1)
            s = random.randint(0,len(greek_corpus_avalible)-1)

            # Pick Symbols in List

            fs = random.randint(0,len(df_dict)-1)
            ss = random.randint(0,len(df_dict)-1)

            # Add Space at Random

            space = random.randint(0,1)

            # Merge
            if space == 1:
                string = eng_corpus_avalible.iloc[f][0] + df_dict.iloc[fs][0] + ' ' + greek_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]
            else:
                string = eng_corpus_avalible.iloc[f][0] + df_dict.iloc[fs][0] + greek_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]

        else:

            # Pick Words in Corpus

            f = random.randint(0,len(greek_corpus)-1)
            s = random.randint(0,len(eng_corpus_avalible)-1)

            # Pick Symbols in List

            fs = random.randint(0,len(df_dict)-1)
            ss = random.randint(0,len(df_dict)-1)

            # Add Space at Random

            space = random.randint(0,1)

            # Merge
            if space == 1:
                string = greek_corpus.iloc[f][0] + df_dict.iloc[fs][0] + ' ' + eng_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]
            else:
                string = greek_corpus.iloc[f][0] + df_dict.iloc[fs][0] + eng_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]

    else:

        # Pick Words in Corpus

        f = random.randint(0,len(greek_corpus)-1)
        s = random.randint(0,len(greek_corpus)-1)

        # Pick Symbols in List

        fs = random.randint(0,len(df_dict)-1)
        ss = random.randint(0,len(df_dict)-1)

        # Add Space at Random

        space = random.randint(0,1)

        # Merge
        if space == 1:
            string = greek_corpus.iloc[f][0] + df_dict.iloc[fs][0] + ' ' + greek_corpus.iloc[s][0] + df_dict.iloc[ss][0]
        else:
            string = greek_corpus.iloc[f][0] + df_dict.iloc[fs][0] + greek_corpus.iloc[s][0] + df_dict.iloc[ss][0]

    if len(string) >= 25:
        string = string[0:25]

    print(string)

recording,! επανεξελέγη'
εκφωνεί{ καταδυναστεύει]
γκρεμιστεί0ΚΑΘΗΓΗΤΡΙΑΣ4
Mbps,5περιγράφει`
συνδυασμένο~ΠΑΘΗΜΑ=
WIG} επαναφέρω9
Clasp,ψαλμοί.
τυπώθηκε}Hairdryers0
λίρα' ΕΦΙΔΡΩΣΗ`
Egmont` επισφαλείς"
ΕΝΕΡΓΟΠΟΙΗΣΕ]ΛΗΣΜΟΝΕΙΣ#
στέφαν3μπορχεσιανό\
άπειρα?(CERN)+
ανταποκρίσεις- ΔΕΥΤΕΡΕΥΟΝ
γκρεμίζεται>ΠΑΛΙΑ[
Uehara\ ασφυκτικά]
κόκκινη"ACSC7
(choose=υπανάπτυξη#
κοριτσάκια7ρεν&
ΣΤΑΥΡΟΥ) σπάσουν\
LDL-C/ ομορφαίνουν7
τριετές"δίκαιους)
pixelated: προσπαθούσαν/
αφαίρεσε\γεωστρατηγική{
ΣΧΕΤΙΖΟΜΕΝΗ[ μεταπώλησης[
ΘΑΝΑΤΩΣΕΙΣ2έμβρυα<
χρεοκοπία'Mykonos'
μακεδονίαν[ αποθηκεύσει3
ΑΡΙΣΤΕΡΟΙ#GOVERNORS~
Landscapes,, εξέλιξής=
ομηρικό!ΓΛΥΤΩΣΑΝ/
Budock@πρωτοδικείο}
δώσεις! Ashfield,#
απορρίψουν> COOKING,]
ΕΠΙΣΚΟΠΗ? ΚΙΤΡΙΝΟΥΣ,
ΟΧΗΜΑ1ΚΕΛΥΦΟΣ~
ΚΑΛΟΠΡΟΑΙΡΕΤΗ2cor-%
αδιαμαρτύρητα(CIM]
ΕΝΕΡΓΗΤΙΚΟΤΗΤΑΣ:ΛΕΙΤΟΥΡΓΟ
ΒΟΥΤΗΞΕΙ7 ΠΡΩΘΥΠΟΥΡΓΟΥΣ(
καταλήξουν:ΙΑΒΑ'
ΑΡΧΑΙΟΤΕΡΑ!ΣΥΓΚΙΝΗΣΗΣ!
ΔΙΕΥΡΥΝΕΤΑΙ*πρωτόγονοι-
testers.&κάποιοι2
έξιν>μαρτηουλι\
ψόφια5 ΥΠΑΡΧΟΥΣΕΣ0
Unlimited's. σοσιαλιστική
ΣΤΑΦΥΛΙ_ ελεφάντης3
αλιευτικού1 ΣΤΡΙΜΩΞΕΙ9

In [62]:
import random

n_sample = 200000

with open('corpus_large.txt', 'w') as z:

    for i in range(n_sample):
        
        # Whether the text is combined by english and greek
        
        greek_eng = random.randint(0,1)
        
        if greek_eng:
            
            # Set english first or greek first
            
            eng_first = random.randint(0,1)
            
            if eng_first:

                # Pick Words in Corpus

                f = random.randint(0,len(eng_corpus_avalible)-1)
                s = random.randint(0,len(greek_corpus_avalible)-1)

                # Pick Symbols in List

                fs = random.randint(0,len(df_dict)-1)
                ss = random.randint(0,len(df_dict)-1)

                # Add Space at Random

                space = random.randint(0,1)

                # Merge
                if space == 1:
                    string = eng_corpus_avalible.iloc[f][0] + df_dict.iloc[fs][0] + ' ' + greek_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]
                else:
                    string = eng_corpus_avalible.iloc[f][0] + df_dict.iloc[fs][0] + greek_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]
            
            else:
                
                # Pick Words in Corpus

                f = random.randint(0,len(greek_corpus_avalible)-1)
                s = random.randint(0,len(eng_corpus_avalible)-1)

                # Pick Symbols in List

                fs = random.randint(0,len(df_dict)-1)
                ss = random.randint(0,len(df_dict)-1)

                # Add Space at Random

                space = random.randint(0,1)

                # Merge
                if space == 1:
                    string = greek_corpus_avalible.iloc[f][0] + df_dict.iloc[fs][0] + ' ' + eng_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]
                else:
                    string = greek_corpus_avalible.iloc[f][0] + df_dict.iloc[fs][0] + eng_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]
            
        else:
            
            # Pick Words in Corpus

            f = random.randint(0,len(greek_corpus_avalible)-1)
            s = random.randint(0,len(greek_corpus_avalible)-1)

            # Pick Symbols in List

            fs = random.randint(0,len(df_dict)-1)
            ss = random.randint(0,len(df_dict)-1)

            # Add Space at Random

            space = random.randint(0,1)

            # Merge
            if space == 1:
                string = greek_corpus_avalible.iloc[f][0] + df_dict.iloc[fs][0] + ' ' + greek_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]
            else:
                string = greek_corpus_avalible.iloc[f][0] + df_dict.iloc[fs][0] + greek_corpus_avalible.iloc[s][0] + df_dict.iloc[ss][0]

        if len(string) >= 25:
            string = string[0:25]

        # print(string)

        z.write(string)
        z.write('\n')